In [8]:
# Work flow:
# Open video file
# Extract frame from video (Extract x number of frames from video?)
# Hash frame
# Compare hash value with other hash values
# Cluster videos based on similarity of hash values

In [9]:
from adjusted_rand_index import rand_index
from sklearn.cluster import KMeans
import os
import io
import math
import PIL
import numpy
from PIL import Image, ImageDraw
from threading import Thread
import numpy as np
from datasketch import MinHash, MinHashLSH
from subprocess import call

DATASET_FOLDER = 'videos/'
TEST_VIDEO = 'ZZTM0X6OON1U.mp4'
IMAGE_FOLDER = "images/"

# Empty dictionary to store video name and hash values
Hashvalues = {}

In [10]:
def get_keyframes_from_video(video):
    for filename in sorted(os.listdir(DATASET_FOLDER))[:10]:
        test_path = DATASET_FOLDER + "/" + filename
        call("""ffmpeg -i """ + DATASET_FOLDER + filename + """ -vf "select='eq(pict_type,PICT_TYPE_I)'" -vsync vfr images/image_%d.png""", shell=True)
        Hashvalues[filename] =  combined_hash_value(IMAGE_FOLDER)
        delete_images()

In [11]:
def delete_images():
    call("rm images/*.png", shell=True)

In [12]:
def convert_to_hash(img_path):
    # Converte img to grayscale & resize img
    img = Image.open(img_path).convert('LA').resize((10,10), PIL.Image.ANTIALIAS)
    numpy_array = numpy.array(img)
    min_hash = MinHash(num_perm=128)
    # Hash img
    for val in numpy_array:
        min_hash.update(val)
        
    return min_hash

In [13]:
def LSH_frames(frames):
    first_frame = frames.pop(0)
    
    for frame in frames:
        first_frame.merge(frame)
    
    return first_frame

In [14]:
def combined_hash_value(imagefolder):
    images = []

    for filename in sorted(os.listdir(imagefolder)):
        hashed_image = convert_to_hash(imagefolder+filename)
        images.append(hashed_image)

    return LSH_frames(images)

In [16]:
get_keyframes_from_video(TEST_VIDEO)

In [17]:
value_list = []

for key in Hashvalues:
    value_list.append(Hashvalues[key].hashvalues)
    
X = np.array(value_list)
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

In [18]:
kmeans.labels_

array([0, 1, 2, 1, 1, 0, 1, 1, 1, 1], dtype=int32)

In [19]:
list(value_list[0])

[848917024,
 737726443,
 704286969,
 41055392,
 694800114,
 838409169,
 388337466,
 110567965,
 363036724,
 101590615,
 540911617,
 150873759,
 1298545514,
 955518194,
 429879970,
 32146942,
 113719762,
 45860336,
 62965964,
 766879834,
 69481422,
 232762981,
 316169793,
 141257415,
 767798310,
 15896916,
 99903574,
 197546216,
 175332131,
 460407794,
 134185777,
 161455122,
 514556473,
 588891260,
 307868170,
 820182822,
 315518132,
 977431640,
 92875461,
 249853397,
 409463548,
 1114055740,
 741819521,
 357275171,
 294083289,
 242332927,
 24618312,
 277893109,
 232395982,
 314637823,
 1196252869,
 173869116,
 1621584903,
 357918799,
 1088712375,
 410693382,
 108042449,
 393195550,
 36599099,
 233061537,
 462245658,
 87203765,
 192307458,
 66450247,
 629357910,
 336499023,
 92464348,
 227644923,
 323641890,
 167236806,
 337227619,
 507089600,
 260384247,
 29385353,
 30735667,
 1409717586,
 324669727,
 774190946,
 354069749,
 166199907,
 20214328,
 130085048,
 566752535,
 69859144,
 59

In [20]:
for key in Hashvalues:    
    if (Hashvalues[key].hashvalues == list(value_list[0])).all():
        print(key)

001T42Y7IZV4.mp4
